In [1]:
import torchvision
import torch
import torchvision.transforms as transforms
import torch.nn.functional as F

from IPython.display import display
import ipywidgets
import ipywidgets.widgets as widgets
import traitlets
from robotpal import Robot, Camera, bgr8_to_jpeg
from robotpal.SCSCtrl import TTLServo

import threading
import time
import cv2
import PIL.Image
import numpy as np

[RobotPal] 서버 시작됨 | WebSocket: 9999, TCP: 9998
[RobotPal] 통신 대기 중... (WS: 9999, TCP: 9998)


[System] WebSocket 프로세서 시작


In [2]:
robot = Robot()
camera = Camera()
TTLServo.servoAngleCtrl(5, 45, 1, 100)

model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 2)

# .pth 파일의 경로를 확인
model.load_state_dict(torch.load('best_steering_model_xy_test_12_17.pth'))

device = torch.device('cuda')
model = model.to(device)
model = model.eval().half()

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()
print('model load success')

lbl1 = ipywidgets.Label(value="areaA :")
areaAlbl = ipywidgets.Label(value="None")
hbox1 = widgets.HBox([lbl1, areaAlbl] )

lbl2 = ipywidgets.Label(value="areaB :")
areaBlbl = ipywidgets.Label(value="None")
hbox2 = widgets.HBox([lbl2,areaBlbl])

lbl3 = ipywidgets.Label(value="self.flag :")
flaglbl = ipywidgets.Label(value="None")
hbox3 = widgets.HBox([lbl3, flaglbl])
vbox1 = widgets.VBox([hbox1, hbox2, hbox3])

image_widget = ipywidgets.Image(format='jpeg', width=224, height=224)
x_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='x')
y_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='y')
steering_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='steering')
speed_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='speed')
vbox2 = widgets.VBox([image_widget, x_slider, steering_slider], layout=widgets.Layout(align_self='center'))
hbox4 = widgets.HBox([vbox2, y_slider, speed_slider], layout=widgets.Layout(align_self='center'))

startBtn = widgets.Button(description="Start", button_style='info')
lbl41 = ipywidgets.Label(value="Find Area : ")
goallbl = ipywidgets.Label(value="None")
hbox5 = widgets.HBox([startBtn,lbl41, goallbl])


#수동 조작용 contorller widget 생성
lbl50 = ipywidgets.Label(value="Manual Controller")

button_layout = widgets.Layout(width='100px', height='80px', align_self='center')
stop_button = widgets.Button(description='stop', button_style='danger', layout=button_layout)
forward_button = widgets.Button(description='forward', layout=button_layout)
backward_button = widgets.Button(description='backward', layout=button_layout)
left_button = widgets.Button(description='left', layout=button_layout)
right_button = widgets.Button(description='right', layout=button_layout)
middle_box = widgets.HBox([left_button, stop_button, right_button], layout=widgets.Layout(align_self='center'))
controls_box = widgets.VBox([lbl50, forward_button, middle_box, backward_button])

#자동 조작용 contorller widget 생성
lbl51 = ipywidgets.Label(value="Auto Controller")
speed_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.25, description='speed gain',disabled = False)
steering_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.2, description='steering gain',disabled = False)
steering_dgain_slider = ipywidgets.FloatSlider(min=0.0, max=0.5, step=0.001, value=0.5, description='steering kd',disabled = False)
steering_bias_slider = ipywidgets.FloatSlider(min=-0.3, max=0.3, step=0.01, value=0.0, description='steering bias',disabled = False)
vbox3 = widgets.VBox([lbl51,speed_gain_slider, steering_gain_slider, steering_dgain_slider, steering_bias_slider])
hbox6 = widgets.HBox([controls_box, vbox3], layout=widgets.Layout(align_self='center'))

display(vbox1, hbox4,hbox5,hbox6)

manual_btnlst = [stop_button, forward_button, backward_button, left_button, right_button]
camera_link = traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

def stop(change):
    robot.stop()
    
def step_forward(change):
    robot.forward(0.4)

def step_backward(change):
    robot.backward(0.4)

def step_left(change):
    robot.left(0.3)
    time.sleep(0.5)
    robot.stop()

def step_right(change):
    robot.right(0.3)
    time.sleep(0.5)
    robot.stop()

stop_button.on_click(stop)
forward_button.on_click(step_forward)
backward_button.on_click(step_backward)
left_button.on_click(step_left)
right_button.on_click(step_right)

# areaA 와 areaB 를 선정
areaA = 'mint'
areaB = 'darkblue'
# 조사한 색상의 이름과 min, max HSV 값, 그리고 회전 각도(angle)를 담은 리스트
colors = [
    # angle: 로봇팔 회전 각도 (단위: 도), 0이면 회전 안 함
    {'name': 'red',      'lower': np.array([3, 217, 214]),   'upper': np.array([4, 222, 218]),    'angle': 0 },
    {'name': 'mint',     'lower': np.array([68, 17, 221]),   'upper': np.array([70, 20, 222]),    'angle': 32.70 }, # 오른쪽으로 32.70도
    {'name': 'blue',     'lower': np.array([106, 212, 207]), 'upper': np.array([108, 223, 216]),  'angle': 29.47 }, # 오른쪽으로 29.47도
    {'name': 'darkblue', 'lower': np.array([120, 201, 182]), 'upper': np.array([120, 206, 189]),  'angle': 29.57 }, # (기존 코드 유지용)
    {'name': 'yellow',   'lower': np.array([27, 85, 195]),   'upper': np.array([30, 97, 221]),    'angle': 0 },
    {'name': 'orange',   'lower': np.array([17, 145, 222]),  'upper': np.array([18, 154, 224]),   'angle': 0 }
]

areaA_color = next((color for color in colors if color['name'] == areaA), None)
areaB_color = next((color for color in colors if color['name'] == areaB), None)

areaAlbl.value = areaA_color['name']
areaBlbl.value = areaB_color['name']

findArea = areaA
goallbl.value = findArea

#frame 크기와 카메라 중심점 좌표 설정
frame_width = 224
frame_height = 224
camera_center_X = int(frame_width/2)
camera_center_Y = int(frame_height/2)

#WorkingAreaFind() 용 리스트와 변수
colorHSVvalueList = []
max_len = 20

#2개 thread 용 객체 변수 생성
roadFinding = None
goalFinding = None

C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


model load success


In [3]:
class WorkingAreaFind(threading.Thread):
    def __init__(self):
        super().__init__()
        self.th_flag=True
        self.imageInput = 0
        
        #area 찾는 순서 변수
        self.flag = 1
        flaglbl.value = str(self.flag)

    # [추가] 화면을 계속 갱신하면서 대기하는 함수
    def wait_and_show(self, seconds, message=""):
        start_time = time.time()
        while time.time() - start_time < seconds:
            # 1. 최신 카메라 프레임 가져오기
            self.imageInput = camera.value
            
            # 2. 메시지가 있다면 화면에 표시
            if message:
                cv2.putText(self.imageInput, message, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
            
            # 3. 위젯 업데이트 (이게 있어야 화면이 안 멈춤)
            image_widget.value = bgr8_to_jpeg(self.imageInput)
            
            # 4. 짧게 대기 (CPU 과부하 방지)
            time.sleep(0.05)
        
    def run(self):
        while self.th_flag:
            self.imageInput = camera.value
            #BGR to HSV
            hsv = cv2.cvtColor(self.imageInput, cv2.COLOR_BGR2HSV)
            #blur
            hsv = cv2.blur(hsv, (15, 15))
                        
            #areaA, areaB Color searching
            areaA_mask = cv2.inRange(hsv, areaA_color['lower'], areaA_color['upper'])
            areaA_mask = cv2.erode(areaA_mask, None, iterations=2)
            areaA_mask = cv2.dilate(areaA_mask, None, iterations=2)
            
            areaB_mask = cv2.inRange(hsv, areaB_color['lower'], areaB_color['upper'])
            areaB_mask = cv2.erode(areaB_mask, None, iterations=2)
            areaB_mask = cv2.dilate(areaB_mask, None, iterations=2)

            # 해당 영역에 대한 윤곽선 따기
            AContours, _ = cv2.findContours(areaA_mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            BContours, _ = cv2.findContours(areaB_mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            #A영역 or B영역을 찾았다면
            if AContours and self.flag == 1:
                self.findCenter(areaA, AContours)
            
            elif BContours and self.flag == 2:
                self.findCenter(areaB, BContours)
            
            #두 영역 모두 못찾았다면, 찾아가는 중이다.
            else:
                cv2.putText(self.imageInput, "Finding...", (0, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5,(255,255,255),1,cv2.LINE_AA)
                image_widget.value = bgr8_to_jpeg(self.imageInput)
            time.sleep(0.1)
    
    #name : A, B 구분용도, Contours 각 영역의 윤곽선 값
    def findCenter(self, name, Contours):  
        global roadFinding, findArea

        c = max(Contours, key=cv2.contourArea)
        ((box_x, box_y), radius) = cv2.minEnclosingCircle(c)

        X = int(box_x)
        Y = int(box_y)
        
        error_Y = abs(camera_center_Y - Y)
        error_X = abs(camera_center_X - X)
        
        if error_Y < 15 and error_X < 15:

            time.sleep(1)

            if 'roadFinding' in globals() and roadFinding:
                roadFinding.paused = True
            robot.stop()

            # 2. 목표 달성 처리 (기존 로직)
            target_reached = False

            #A영역이 가까이 오게됨
            if name == areaA and self.flag == 1:
                self.flag = 2
                findArea = areaB
                goallbl.value = findArea
                
                areaAlbl.value = areaA + " Goal!"
                flaglbl.value = str(self.flag)
                target_reached = True
                
            #B영역이 가까이 오게됨
            elif name == areaB and self.flag == 2:              
                self.flag = 1       
                findArea = areaB
                goallbl.value = findArea
                
                areaBlbl.value = areaB + " Goal!"
                flaglbl.value = str(self.flag)   
                target_reached = True

            # 3. 도착했다면 3초간 대기 후 주행 재개
            if target_reached:
                
                # 1) colors 리스트에서 현재 감지된 색상(name)의 정보 찾기
                # next() 함수를 사용하여 colors 리스트에서 name이 일치하는 첫 번째 딕셔너리를 가져옵니다.
                current_color_info = next((c for c in colors if c['name'] == name), None)
                
                # 각도 가져오기 (정보가 없으면 기본값 0)
                target_angle = current_color_info['angle'] if current_color_info else 0
                
                # # 화면에 상태 표시
                # cv2.putText(self.imageInput, f"OCR Scanning... {target_angle} deg", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
                # image_widget.value = bgr8_to_jpeg(self.imageInput)

                # 각도가 0이 아닐 때만 회전 동작 수행 (선택 사항)
                if target_angle != 0:
                    # 2) 로봇팔 회전
                    TTLServo.servoAngleCtrl(1, target_angle, 1, 100)
                    self.wait_and_show(1.5, f"Rotating {target_angle}...")

                    # 3) OCR 수행 구간 (대기)
                    print(f"[{name}] OCR capturing at {target_angle} degrees...")
                    self.wait_and_show(2, "OCR Capturing...")

                    # 4) 로봇팔 정면 복귀
                    TTLServo.servoAngleCtrl(1, 0, 1, 100)
                    self.wait_and_show(1.5, "Returning...")
                else:
                    # 각도가 0이면(정면이면) 그냥 잠시 대기
                    print(f"[{name}] OCR capturing at frontal view...")
                    time.sleep(5) # 회전 시간이 없으므로 대기 시간 조정
                
                # 5) 주행 재개
                if 'roadFinding' in globals() and roadFinding:
                    roadFinding.paused = False
                
        image_widget.value = bgr8_to_jpeg(self.imageInput)
        
    def stop(self):
        self.th_flag = False
        robot.stop()

class RobotMoving(threading.Thread):
    def __init__(self):
        super().__init__()
        self.th_flag = True
        self.paused = False
        self.angle = 0.0
        self.angle_last = 0.0
        
    def run(self):
        while self.th_flag:

            if self.paused:
                robot.stop()
                time.sleep(0.1)
                continue


            image = camera.value
            xy = model(self.preprocess(image)).detach().float().cpu().numpy().flatten()
            x = xy[0]
            y = (0.5 - xy[1]) / 2.0
            
            x_slider.value = x
            y_slider.value = y
            
            #인공지능 무인운반로봇(AGV)의 속도 표시
            speed_slider.value = 0.23
            
            image_widget.value = bgr8_to_jpeg(image)
            
            #조향값 계산
            self.angle = np.arctan2(x, y)
            
            if not self.th_flag:
                break
            #PID 제어를 이용한 모터 제어
            pid = self.angle * 0.2 + (self.angle - self.angle_last) * 0.5
            self.angle_last = self.angle

            #슬라이더에 표시
            steering_slider.value = pid

            robot.left_motor.value = max(min(speed_slider.value + steering_slider.value, 1.0), -0.9)
            robot.right_motor.value =  max(min(speed_slider.value - steering_slider.value, 1.0), -0.9)
            time.sleep(0.1)
        robot.stop()
    
    def preprocess(self, image):
        image = PIL.Image.fromarray(image)
        image = transforms.functional.to_tensor(image).to(device).half()
        image.sub_(mean[:, None, None]).div_(std[:, None, None])
        return image[None, ...]

    def stop(self):
        self.th_flag = False
        robot.stop()

def start(change):
    global camera_link, goalFinding, roadFinding
    #Manual -> Auto
    if startBtn.description == "Start" :
        camera_link.unlink()
        goalFinding = WorkingAreaFind()
        goalFinding.start()
        roadFinding = RobotMoving()
        roadFinding.start()

        startBtn.button_style = "warning" 
        startBtn.description = "Stop"
    
    #Auto -> Manual
    elif startBtn.description == "Stop":
        roadFinding.stop()
        goalFinding.stop()
        camera_link = traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)
            
        startBtn.button_style = "info"
        startBtn.description = "Start"

startBtn.on_click(start)

In [ ]:
time.sleep(0.1)
robot.stop()
camera.stop()

print('End')